## Install and import depdencies 

In [1]:
!conda install pytorch torchvision torchaudio pytorch-cuda=12.1 -c pytorch -c nvidia

Channels:
 - pytorch
 - nvidia
 - defaults
 - anaconda
Platform: win-64
Solving environment: ...working... done

# All requested packages already installed.



In [2]:
!pip install ultralytics

In [3]:
from ultralytics import YOLO
import torch
import numpy as np
import cv2
from matplotlib import pyplot as plt

## Prep training materials

In [4]:
import os
import shutil
import random

Please make sure all images are labeled using YOLO format.

INSTRUCTIONS:
1. Create a folder named "data" OUTSIDE of the local repo. This folder will contain all your data.
2. Create a subdirectory inside the data folder and add one folder named "unsplit". Add your images and labels folders to this subdirectory.
   I have included an exmaple directory inside the RSYOLOV8 folder. This is how your data folder should be set up before running the next part of the code.
3. Please go into the config.yaml file and set the "path" variable to the ABSOLUTE PATH of the data folder.
   

In [5]:
# Set the path to the data folder below as well.
data_dir = "D:/RecieptScanner\data"

In [6]:
# Define paths
unsplit_dir = os.path.join(data_dir, 'unsplit')
images_dir = os.path.join(unsplit_dir, 'images')
labels_dir = os.path.join(unsplit_dir, 'labels')

In [7]:
# New directories
train_dir = os.path.join(data_dir, 'train')
val_dir = os.path.join(data_dir, 'val')
test_dir = os.path.join(data_dir, 'test')

In [8]:
# Create new directories
for split_dir in [train_dir, val_dir, test_dir]:
    os.makedirs(os.path.join(split_dir, 'images'), exist_ok=True)
    os.makedirs(os.path.join(split_dir, 'labels'), exist_ok=True)

In [9]:
train_ratio = 0.7
val_ratio = 0.2
test_ratio = 0.1

In [10]:
# get unsplit images and labels
images = [f for f in os.listdir(images_dir) if f.endswith('.jpg')]
labels = [f for f in os.listdir(labels_dir) if f.endswith('.txt')]

In [11]:
# Ensure there is a label for each image
images = [img for img in images if f'{os.path.splitext(img)[0]}.txt' in labels]

In [12]:
random.shuffle(images)

In [13]:
# Calculate split sizes
total_images = len(images)
train_size = int(total_images * train_ratio)
val_size = int(total_images * val_ratio)
test_size = total_images - train_size - val_size

In [14]:
# Split images and labels
train_images = images[:train_size]
val_images = images[train_size:train_size + val_size]
test_images = images[train_size + val_size:]

In [15]:
# Function to move files to appropriate directories
def move_files(file_list, source_img_dir, source_lbl_dir, target_dir):
    for file_name in file_list:
        img_source_path = os.path.join(source_img_dir, file_name)
        lbl_source_path = os.path.join(source_lbl_dir, f'{os.path.splitext(file_name)[0]}.txt')
        
        img_target_path = os.path.join(target_dir, 'images', file_name)
        lbl_target_path = os.path.join(target_dir, 'labels', f'{os.path.splitext(file_name)[0]}.txt')
        
        shutil.copy(img_source_path, img_target_path)
        shutil.copy(lbl_source_path, lbl_target_path)

In [16]:
# Move files to train, val, test directories
move_files(train_images, images_dir, labels_dir, train_dir)
move_files(val_images, images_dir, labels_dir, val_dir)
move_files(test_images, images_dir, labels_dir, test_dir)

# You are now ready to train!

## Select model architecture

In [17]:
model = YOLO('yolov8n.yaml')  # or 'yolov8s.pt' for pretrained weights

## Train the model

In [18]:
# data: Specifies the path to the dataset configuration file
# imgsz: Sets the image size
# batch: Sets the batch size
# epochs: Sets the number of epochs
# workers: Sets the number of data loader workers ( i have no idea what this does )
model.train(data='config.yaml', epochs=3)

New https://pypi.org/project/ultralytics/8.2.31 available  Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.28  Python-3.11.9 torch-2.3.1 CUDA:0 (NVIDIA GeForce RTX 2060, 6144MiB)
engine\trainer: task=detect, mode=train, model=yolov8n.yaml, data=config.yaml, epochs=3, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train9, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=Fal

train: Scanning D:\RecieptScanner\data\train\labels... 4 images, 0 backgrounds, 0 corrupt: 100%|██████████| 4/4 [00:00<

train: New cache created: D:\RecieptScanner\data\train\labels.cache



val: Scanning D:\RecieptScanner\data\val\labels... 1 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1/1 [00:00<00:0

val: New cache created: D:\RecieptScanner\data\val\labels.cache


...


3 epochs completed in 0.005 hours.
Optimizer stripped from runs\detect\train9\weights\last.pt, 6.2MB
Optimizer stripped from runs\detect\train9\weights\best.pt, 6.2MB

Validating runs\detect\train9\weights\best.pt...
Ultralytics YOLOv8.2.28  Python-3.11.9 torch-2.3.1 CUDA:0 (NVIDIA GeForce RTX 2060, 6144MiB)
YOLOv8n summary (fused): 168 layers, 3006233 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<0

                   all          1          6          0          0          0          0


Speed: 1.5ms preprocess, 18.1ms inference, 0.0ms loss, 7.0ms postprocess per image
Results saved to runs\detect\train9


AttributeError: 'DetMetrics' object has no attribute 'curves_results'. See valid attributes below.

    This class is a utility class for computing detection metrics such as precision, recall, and mean average precision
    (mAP) of an object detection model.

    Args:
        save_dir (Path): A path to the directory where the output plots will be saved. Defaults to current directory.
        plot (bool): A flag that indicates whether to plot precision-recall curves for each class. Defaults to False.
        on_plot (func): An optional callback to pass plots path and data when they are rendered. Defaults to None.
        names (tuple of str): A tuple of strings that represents the names of the classes. Defaults to an empty tuple.

    Attributes:
        save_dir (Path): A path to the directory where the output plots will be saved.
        plot (bool): A flag that indicates whether to plot the precision-recall curves for each class.
        on_plot (func): An optional callback to pass